# Get DHS Spatial Data
This notebook will walk you through the steps for downloading DHS Spatial Boundaries by Survey Id, and publishing them to your ArcGIS Online Organization. 

It will also output a base configuration in JSON format that you can use in the "Join CSV SDG Data to Geography" python notebook as part of configuring your SDG Hub Template.

### Import the needed python libraries

In [ ]:
import os
import requests
import json
import getpass
from os import mkdir
from os.path import exists
from arcgis import GIS
from arcgis.geoprocessing import import_toolbox
import ipywidgets as widgets
from urllib.request import urlopen
from ipywidgets import interact

### Select your Country
Run this cell to see a drop down list of available countries, then make your selection.

After you have made your selection, move on to the next cell and do not re-execute this cell.

In [ ]:
countries = json.loads(urlopen('http://api.dhsprogram.com/rest/dhs/countries?f=json').read())
id = 0
countryList = []
for country in countries['Data']:
  country_code = country['DHS_CountryCode']
  country_name = country['CountryName']
  # countryList.append((country_name, country_code))
  countryList.append(country_name)
  id += 1
  
selected_country = None
def f(Country):
  global selected_country
  # selectedCountry = Country
  # return Country
  for item in countries['Data']:
    if item['CountryName'] == Country:
      selected_country = item['DHS_CountryCode']
      return selected_country

countries_cb = widgets.Combobox(
  # placeholder='Choose Country',
  options=countryList,
  description='Country',
  ensure_option=True,
  disabled=False
)

interact(f, Country=countries_cb)

### Select a survey for your selected country
After you have made your selection, move on to the next cell and do not re-execute this cell.

In [ ]:
surveys = json.loads(urlopen(f'https://previewapi.dhsprogram.com/rest/dhs/surveys/{selected_country}').read())
survey_list = []
for survey in surveys['Data']:
  survey_id = survey['SurveyId']
  survey_list.append(survey_id)

surveys_sm = widgets.SelectMultiple(
  options=survey_list,
  description='Surveys',
  disabled=False
)

selected_surveys = []
def sm_f(SurveyName):
  return SurveyName
  
interact(sm_f, SurveyName=surveys_sm)

Reformat the information

In [ ]:
# [['ZM', ['ZM2018DHS']]]
user_input = [[selected_country, list(surveys_sm.value)]]
user_input

### Setup the GP Service for downloading the Country Shapefiles
DHS provides shapefiles for each corresponding survey. This GP Service is used to download the shapefiles

In [ ]:
gp_url = 'https://gis.dhsprogram.com/arcgis/rest/services/Tools/DownloadSubnationalData/GPServer/downloadSubNationalBoundaries'
tbx = import_toolbox(gp_url)

Helper function to build reference information for each survey

In [ ]:
def get_survey_ids():
  country_code = user_input[0][0]
  surveys = [sid for sid in user_input[0][1]]
  survey_ids = ','.join(surveys)
  req_url = f'https://api.dhsprogram.com/rest/dhs/surveys/{country_code}?surveyIds={survey_ids}'

  res = requests.get(req_url).json()
  if res['RecordsReturned'] == 0:
    raise(f'no survey ids found for URL :: {req_url}')

  survey_info = [{'SurveyNum': rec['SurveyNum'], 'CountryCode': country_code, 'SurveyId': surveys[a]} for a,rec in enumerate(res['Data'])]

  return survey_info

Helper function to make directories

In [ ]:
def clear_and_make_dir(path):
  if exists(path):
      from shutil import rmtree
      rmtree(path)
  mkdir(path)

Set the current working directory for downloading the shapefiles

In [ ]:
# if you are running this notebook in ArcGIS Online
base_directory = '/arcgis/home'

In [ ]:
download_path = f'{base_directory}/shapefiles'
clear_and_make_dir(f'{download_path}')

### Download a shapefile for each survey

In [ ]:
survey_info = get_survey_ids()
spatial_format = 'shp'

for survey in survey_info:
    result = tbx.download_sub_national_boundaries(survey_ids=survey['SurveyNum'], spatial_format=spatial_format)
    country_code = survey['CountryCode']
    survey_id = survey['SurveyId']
    file_name =  f'{country_code}_{survey_id}'
    file_path = download_path + f'/{file_name}'

    result.result.download(save_path=file_path)

    rurl = result.result.url
    start = rurl.rindex('/')+1
    zip_file_name = rurl[start:]
    os.rename(os.path.join(file_path, zip_file_name), os.path.join(file_path,f'{file_name}.zip'))
    survey['shapefile_path'] = file_path + f'/{file_name}.zip'

### Connect to your ArcGIS Online Organization

In [ ]:
# use if you are running in a hosted notebook in ArcGIS Online
gis = GIS('home')

# use if you are running a notebook on your local machine in your own python environment
# username = input("Enter a built-in user name with access to SDG Hub Template items: ")
# password = getpass.getpass('Enter your password: ')
# gis = GIS('https://www.arcgis.com', username, password)

### Publish each shapefile to ArcGIS Online
This cell will also build a `geography_config` variable that can be used in the "Join CSV SDG Data to Geography" python notebook.

In [ ]:
geography_config = []
for info in survey_info:
    shp_path = info['shapefile_path']
    survey_name = info['SurveyId']
    print(f'uploading shapefile for :: {survey_name}')

    item_properties = {
    'title': survey_name
    }

    shp_item = gis.content.add(item_properties=item_properties, data=info['shapefile_path'])
    print(f'publishing shapefile for :: {survey_name} ...')
    published_shp = shp_item.publish()
    print(f'successfully published shapefile for :: {survey_name}')

    layer_counter = 0
    country_code = info['CountryCode']
    for layer in published_shp.layers:
        config = {
          'name': f'{country_code}_{survey_name}_{layer_counter}',
          'itemId': published_shp.id,
          'layerId': layer_counter,
          'joinField': {
            'name': 'REG_ID',
            'type': 'string'
          }
        }

        geography_config.append(config)
        layer_counter += 1
        
print('done publishing shapefiles')

### Print out the `geography_config`
Execute this cell and then copy and paste the output in your "Join CSV SDG Data to Geography" python notebook.

In [ ]:
print('geography_config = ' + json.dumps(geography_config, indent=4))